<a href="https://colab.research.google.com/github/LuigiSigillo/CartoonGAN/blob/main/CartoonGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [35]:
from google.colab import drive
import os
import json
import re
import cv2
import numpy as np
drive.mount('/content/drive')
#!unzip /content/drive/My\ Drive/NN/spirited_away.zip -d /content/drive/My\ Drive/NN/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls /content/drive/My\ Drive/NN/

paprika			  paprika.zip	     your_name
paprika_resized		  spirited_away      your_name_resized
paprika_resized_smoothed  spirited_away.zip  your_name.zip


# 1 - Image Preprocessing

### 1.1 - Resizing images


In [ ]:
from PIL import Image
import os, sys



def resize(path):
    for item in os.listdir(path):
            im = Image.open(path+item)
            f, e = os.path.splitext(item)
            imResize = im.resize((256,256), Image.ANTIALIAS)
            print(f)
            imResize.save(path+"_resized/" + f + ' resized.jpg', 'JPEG', quality=90)

resize('/content/drive/My Drive/NN/your_name')

### 1.2 Apply canny, dilate edge and gaussian

In [ ]:
import os
import cv2
from PIL import Image, ImageFilter
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
def edge_smoothing(cartoon_images_filename, smoothed_images_filename):
    print("Edge-smoothing of ", cartoon_images_filename)
    origin = cv2.imread(cartoon_images_filename)
    edges = createEdgesOverlay(origin)
    result = overlayEdges(edges, origin)
    #show_images(origin, edges, result)
    result.save(smoothed_images_filename, "JPEG")

def overlayEdges(edges, origin):
    background = transformFromCV2ToPillowImageFormat(origin)
    background.paste(edges, (0, 0), edges)
    background = background.convert("RGB")
    return background

def transformFromCV2ToPillowImageFormat(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGBA)
    return Image.fromarray(img)

def createEdgesOverlay(origin):
    edges = cv2.Canny(origin, 30, 300, 3) 
    edges = cv2.dilate(edges, (3, 3))
    edges = cv2.bitwise_not(edges)
    edges = transformFromCV2ToPillowImageFormat(edges)
    makeWhiteBackgroundTransparent(edges)
    edges = edges.filter(ImageFilter.GaussianBlur) #do blurring here because doing it before making background transparent results in white halo

    return edges

def makeWhiteBackgroundTransparent(img):
    datas = img.getdata()
    newData = []
    for item in datas:
        if item[0] == 255 and item[1] == 255 and item[2] == 255:
            newData.append((255, 255, 255, 0))
        else:
            newData.append(item)
    img.putdata(newData)

def show_images(img,edges,result):
    plt.subplot(131),plt.imshow(img)
    plt.title('Original Image'), plt.xticks([]), plt.yticks([])

    plt.subplot(132),plt.imshow(edges)
    plt.title('Edge Image'), plt.xticks([]), plt.yticks([])
    
    plt.subplot(133),plt.imshow(result)
    plt.title('Result Image'), plt.xticks([]), plt.yticks([])

    plt.show()
    



path_resized = "/content/drive/My Drive/NN/spirited_away_resized/"
path_smoothed = "/content/drive/My Drive/NN/spirited_away_resized_smoothed/"


for filename in os.listdir(path_resized):
  #filename='scene43626 resized.jpg'
  f = filename.split(" ")[0] + " smoothed"
  cartoon_images_filename = path_resized + filename
  smoothed_images_filename = path_smoothed + f
  edge_smoothing(cartoon_images_filename, smoothed_images_filename)



# Valerio